In [1]:
import pandas as pd 
import json
import matplotlib.pyplot as plt 

#Graph network imports
from graphframes import *
from pyspark import *
from pyspark.sql import *
import numpy as np
from pyspark.ml.linalg import *
from pyspark.ml.linalg import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *

from pyspark.sql.functions import udf #user defined function
from pyspark.sql.types import * #Import types == IntegerType, StringType etc.

import nltk

In [2]:
#import statements
from pyspark.sql import SparkSession

#create Spark session
spark = SparkSession.builder.enableHiveSupport().appName('modeling_file').getOrCreate()

conf = spark.sparkContext#._conf.setAll([('spark.executor.memory', '15g'), 
#                                         ('spark.app.name', 'Final_project_read_write'), 
#                                         ('spark.executor.cores', '10'), 
#                                         ('spark.cores.max', '10'), 
#                                         ('spark.driver.memory','20g')])

#print spark configuration settings
#spark.sparkContext.getConf().getAll()

In [9]:
!ls ~/scratch-midway2/big_data_project

02_modeling_rcc.py  modeling_data2	   setup.sh
encodedData	    multiple2.batch	   unzipped_files
logs		    multiple_66245414.err  word2vec_model_v50_min500_ws5_2.0
manifest2.txt	    multiple_66245414.out  work
manifest.txt	    scimagojr_2017.csv


In [5]:
#modeling_data = spark.read.parquet('modeling_data')
modeling_data = spark.read.parquet('big_data_project/modeling_data2')

In [6]:
modeling_data.printSchema()

root
 |-- id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- inCitations_count: integer (nullable = true)
 |-- outCitations_count: integer (nullable = true)
 |-- abstract_wcount: integer (nullable = true)
 |-- title_wcount: integer (nullable = true)
 |-- abstract_tfidf: vector (nullable = true)
 |-- title_tfidf: vector (nullable = true)
 |-- SJR: float (nullable = true)
 |-- author_count: integer (nullable = true)
 |-- fieldsOfStudyVec: vector (nullable = true)
 |-- sourcesVec: vector (nullable = true)
 |-- title_wordVectors: vector (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
modeling_data.count()

11300912

In [7]:
modeling_data.dropna().count()

11300912

In [7]:
#split data into train and test
train_df, test_df = modeling_data.select('id','inCitations_count', 'features').randomSplit([0.8, 0.2], seed=42)
train_df.show(1)

+--------------------+-----------------+--------------------+
|                  id|inCitations_count|            features|
+--------------------+-----------------+--------------------+
|000147021677b6ba9...|                0|(526947,[0,2,3,37...|
+--------------------+-----------------+--------------------+
only showing top 1 row



In [8]:
print('Train Length = ', train_df.count())
print('Test Length = ', test_df.count())

Train Length =  9042363
Test Length =  2258549


In [15]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_with_vectors = train_df.select(
    train_df["id"], train_df['inCitations_count'],
    list_to_vector_udf(train_df["features"]).alias("features"))

In [16]:
df_with_vectors.show(5)

Py4JJavaError: An error occurred while calling o249.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 511, hd05.rcc.local, executor 21): ExecutorLostFailure (executor 21 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits.  1.5 GB of 1.5 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead or disabling yarn.nodemanager.vmem-check-enabled because of YARN-4714.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1890)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1878)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at sca

In [12]:
from pyspark.ml.regression import GeneralizedLinearRegression, GBTRegressor

glr_tweedie =  GeneralizedLinearRegression(family="poisson", link = 'log', 
                                           maxIter=10,
                                           fitIntercept = True,
                                           labelCol = "inCitations_count",
                                           weightCol = "features",
                                           regParam=0.3)

In [13]:
glr_tweedie_model = glr_tweedie.fit(train_df)

IllegalArgumentException: 'requirement failed: Column features must be of type numeric but was actually of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.'

In [ ]:
%%time
from pyspark.ml.regression import LinearRegression

#Elastic Net
lr = LinearRegression(featuresCol = 'features', labelCol='inCitations_count', regParam=0.3, maxIter=10)
lrm = lr.fit(train_df)

#coefficients
print("Coefficients: " + str(lrm.coefficients))
print("Intercept: " + str(lrm.intercept))

#model summary
print("RMSE: %f" % lrm.summary.rootMeanSquaredError)
print("r2: %f" % lrm.summary.r2)

#p-values are not provided in this model for the solver being used
#print("pValues: " + str(lrm.summary.pValues))

In [ ]:
#model summary
print("RMSE: %f" % lrm.summary.rootMeanSquaredError)
print("r2: %f" % lrm.summary.r2)

In [ ]:
# saving:
lrm.save('lrm_model_ss')

# # loading:
# from pyspark.ml.regression import LinearRegressionModel
# model = LinearRegressionModel.load('lrm_model_ss')

In [25]:
#model summary
print("RMSE: %f" % lrm.summary.rootMeanSquaredError)
print("r2: %f" % lrm.summary.r2)

RMSE: 30.617903
r2: 0.067576


In [20]:
#make predictions
predictions = lrm.transform(test_df)

In [21]:
from itertools import chain
attrs = sorted(
    (attr["idx"], attr["name"]) for attr in (chain(*predictions
        .schema[lrm.summary.featuresCol]
        .metadata["ml_attr"]["attrs"].values())))

In [ ]:
attrs

In [23]:
len(attrs)

526477

In [24]:
#[(name, lrm.summary.pValues[idx]) for idx, name in attrs]

from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="inCitations_count", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 25.949
MSE: 673.376
MAE: 5.579
r2: -0.013


In [ ]:
%%time
from pyspark.ml.regression import RandomForestRegressor

# Set parameters for the Random Forest.
rfr = RandomForestRegressor(maxDepth=5, numTrees=20, labelCol="inCitations_count", predictionCol="prediction")

# Fit the model to the data.
rfrm = rfr.fit(train_df)

In [ ]:
# Given a dataset, predict each point's label, and show the results.
predictions = rfrm.transform(test_df)

Exercise: Build a feature importance selector
Reference:
https://www.timlrx.com/2018/06/19/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator/